In [1]:
import sys
sys.path.append('/home/zhw272/programming/pipeline_utility')
from notebooks.Will.toolbox.IOs.get_transfromed_com import *
from notebooks.Will.toolbox.IOs.get_kui_com import *

In [3]:
prepi = 'DK39'
billi_dk52_dict = get_transformed_com_dict(prepi)
kui_com = get_kui_dk52_com()

In [7]:
billi_dk52_dict.keys() == kui_com.keys()

True